# Linear Regression Data Prep

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import ml_utils as mt
import pandas as pd 
import numpy as np

In [ ]:
data_train=pd.read_csv(r'./loan_data_train.csv')

In [ ]:
data_train.head()

In [ ]:
def dtr(orig_col):
    
    mod_col=orig_col.str.replace('%','')
    mod_col=pd.to_numeric(mod_col,errors='coerce')
    
    return mod_col
    

def fico(orig_col):
    k=orig_col.str.split('-',expand=True)
    
    for i in [0,1]:
        k[i]=pd.to_numeric(k[i],errors='coerce')
    
    mod_col=0.5*(k[0]+k[1])
    
    return mod_col
    

def el(orig_col):
    
    inter_col=orig_col.str.replace('10+ years','10',regex=False)
    inter_col=inter_col.str.replace('< 1 year','0',regex=False)
    inter_col=inter_col.str.replace('years','').str.replace('year','')
    
    mod_col=pd.to_numeric(inter_col,errors='coerce')
    
    return mod_col


cat_to_dummies=['Loan.Length','Loan.Purpose','State','Home.Ownership']
cat_to_num=['Amount.Requested','Open.CREDIT.Lines','Revolving.CREDIT.Balance']
simple_num=['Monthly.Income','Inquiries.in.the.Last.6.Months']
custom_func_dict={'Debt.To.Income.Ratio':dtr,'FICO.Range':fico,'Employment.Length':el}

dp=mt.DataPipe(cat_to_dummies=cat_to_dummies,
                 cat_to_num=cat_to_num,
                 simple_num=simple_num,
                 custom_func_dict=custom_func_dict)

In [ ]:
dp.fit(data_train)

In [ ]:
x_train=dp.transform(data_train)

In [ ]:
y_train=data_train['Interest.Rate'].str.replace('%','').astype(float)

# Estimating Model Coefficients with Closed Form Solution

In [ ]:
x_la=x_train.copy()

In [ ]:
x_la.insert(0,'constant',1)

In [ ]:
x_t_x=np.dot(x_la.T,x_la)

x_t_x_inv=np.linalg.inv(x_t_x)

In [ ]:
y_t_x=np.dot(y_train.T,x_la)

In [ ]:
y_t_x

w_la=np.dot(x_t_x_inv,y_t_x)

w_la

# Estimating Model Coefficient with Gradient Descent

In [ ]:
def mypred(x,w):
    
    y_hat=x@w
    return(y_hat)


def myerror(y,x,w):
    
    y_hat=mypred(x,w)
    errors=y-y_hat
    return(errors)


def mycost(y,x,w):
    
    errors=myerror(y,x,w)
    
    cost=errors.T@errors
    
    return(cost)


def gradient(y,x,w):
    
    errors=myerror(y,x,w)
    grad=-x.T@errors/x.shape[0]
    return(grad)

def my_lr_sgd(y,x,learning_rate,num_steps):
    
    weights=np.zeros(x.shape[1])
    decay_rate = 0.9
    prev_cost = float('inf')
 
    for i in np.arange(num_steps):
        rand_ind=np.random.choice(range(x.shape[0]),100)
        y_sub=y[rand_ind]
        x_sub=x.iloc[rand_ind,:]
        
        gd=gradient(y_sub,x_sub,weights)
        
        weights -= learning_rate*gd
        
        curr_cost = mycost(y, x, weights)

        # Stop if converged
        if np.abs(prev_cost - curr_cost) < 1e-6:
            print(f"Converged at iteration {i}")
            break
        
        
        if i%20000==0:
            print(i,curr_cost)
            learning_rate*=decay_rate
            
        prev_cost = curr_cost
            
    return weights

In [ ]:
mycost(y_train,x_la,w_la)

$$
\begin{align*}
w_j&=\frac{{w_j}'}{\sigma_j} \quad \forall j \in \{1,2,\cdots,p\}\\
w_0&={w_0}'-\sum_{j=1}^p\frac{\mu_j{w_j}'}{\sigma_j}
\end{align*}
$$

In [ ]:
def convert_to_non_standardized_weights(scaler, standardized_weights):
    # Extract mean and scale (standard deviation) from the StandardScaler object
    means = scaler.mean_
    scales = scaler.scale_

    # Initialize the array to store the non-standardized weights
    non_standardized_weights = np.zeros_like(standardized_weights)

    # Compute the weights for the non-standardized data
    # w_j_non_standardized = w_j_standardized / scale_j for j > 0
    non_standardized_weights[1:] = standardized_weights[1:] / scales

    # Adjust the intercept term (w_0)
    w0_adjustment = np.sum((means * standardized_weights[1:]) / scales)
    non_standardized_weights[0] = standardized_weights[0] - w0_adjustment

    return non_standardized_weights

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(x_train)
x_sd=pd.DataFrame(scaler.transform(x_train),columns=x_train.columns)
x_sd.insert(0,'constant',1)

In [ ]:
w_sgd=my_lr_sgd(y_train,x_sd,.01,500000)
# see how the cost improves fast initially and then as we reach towards the optimal point
# progresss slows down, this goes on for awhile , lets be patient for 10-15 mins

In [ ]:
w_sgd=convert_to_non_standardized_weights(scaler,w_sgd)

In [ ]:
mycost(y_train,x_la,w_sgd)

In [ ]:
# you can see that we have been able to reach cost levels almost as good as closed form solution
# try other optimisers that we discussed and see how those fare 

In [ ]:
list(zip([1]+list(x_train.columns),list(w_la),list(w_sgd)))

# sklearn estimates

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
sk_lr=LinearRegression()

In [ ]:
sk_lr.fit(x_train,y_train)

In [ ]:
w_sk=[sk_lr.intercept_]+list(sk_lr.coef_)

In [ ]:
list(zip([1]+list(x_train.columns),list(w_la),list(w_gd),w_sk))

# Lasso [Linear Regression with $l_1$ and $l_2$ Penalty ] with gradient descent

code below can be used with some fixed value for penalty parameter $\alpha$ with either $l_1$ or $l_2$ penalty at a time , do experiment around with it.

In [ ]:
def mypred(x, w):
    y_hat = x @ w
    return y_hat

def myerror(y, x, w):
    y_hat = mypred(x, w)
    errors = y - y_hat
    return errors

def mycost(y, x, w, alpha=0.1, penalty='l2'):
    """
    Computes the cost function with either L1 or L2 regularization.

    Parameters:
    - y: actual values
    - x: input data
    - w: weights
    - alpha: regularization strength
    - penalty: 'l1' for Lasso (L1) or 'l2' for Ridge (L2)

    Returns:
    - cost: the regularized cost function value
    """
    errors = myerror(y, x, w)
    
    # Basic cost (squared error)
    cost = errors.T @ errors / (2 * x.shape[0])
    
    # Apply either L1 or L2 penalty
    if penalty == 'l1':
        l1_penalty = np.sum(np.abs(w))
        total_cost = cost + alpha * l1_penalty
    elif penalty == 'l2':
        l2_penalty = np.sum(w ** 2)
        total_cost = cost + alpha * l2_penalty
    else:
        raise ValueError("Invalid penalty type. Use 'l1' or 'l2'.")
    
    return total_cost

def gradient(y, x, w, alpha=0.1, penalty='l2'):
    """
    Computes the gradient with either L1 or L2 regularization.

    Parameters:
    - y: actual values
    - x: input data
    - w: weights
    - alpha: regularization strength
    - penalty: 'l1' for Lasso (L1) or 'l2' for Ridge (L2)

    Returns:
    - grad: the gradient of the cost function with L1 or L2 regularization
    """
    errors = myerror(y, x, w)
    
    # Gradient of the error term
    grad = -x.T @ errors / x.shape[0]
    
    # Apply either L1 or L2 regularization gradient
    if penalty == 'l1':
        l1_grad = np.sign(w)  # Subgradient for L1
        total_grad = grad + alpha * l1_grad
    elif penalty == 'l2':
        l2_grad = 2 * w
        total_grad = grad + alpha * l2_grad
    else:
        raise ValueError("Invalid penalty type. Use 'l1' or 'l2'.")
    
    return total_grad

## Ridge and Lasso [linear regression with $l_2$ and $l_1$ penalty] with sklearn 

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
alphas_ridge=np.linspace(1,100,100)

these are the values of $\alpha$ we want to start our experiment with. `GridSearchCV` scores all these values with cross validation and we can extract with function `ml_utils.report` which value combination scores the best. We need to pass the parameters values that we want to experiment with as a dictionary

In [ ]:
params_ridge={'alpha':alphas_ridge} # key values here need to match exactly as the argument in the modeling functions

In [ ]:
lr_ridge=Ridge() # this is the model for which we want to experiment with parameter values 

In [ ]:
gs_ridge=GridSearchCV(lr_ridge,
               param_grid=params_ridge,
               cv=10, # 10 fold cross validation 
               scoring='neg_mean_absolute_error', # all models are scored with this criterion
               verbose=20, # higher number should print more info while fitting [currently doesnt work well with jupyter]
               n_jobs=-1) # allows for parallel processing

In [ ]:
gs_ridge.fit(x_train,y_train)

In [ ]:
mt.report(gs_ridge.cv_results_,5)

we see that the best value for alpha is `49` , we can further finetune this is by exploring the close range around this value

In [ ]:
params_ridge={'alpha':np.linspace(48,50,25)} 

In [ ]:
params_ridge

In [ ]:
gs_ridge=GridSearchCV(lr_ridge,
               param_grid=params_ridge,
               cv=10, 
               scoring='neg_mean_absolute_error',
               verbose=20, 
               n_jobs=-1) 

In [ ]:
gs_ridge.fit(x_train,y_train)

In [ ]:
mt.report(gs_ridge.cv_results_,5)

we can build the model for best parameter values separately

In [ ]:
ridge_final=Ridge(**{'alpha': 49})

In [ ]:
ridge_final.fit(x_train,y_train)

look at the weights estimate , none of them have been suppressed to exactly zero , but you will do see many have been suppressed b ya large factor

In [ ]:
ridge_wt_comparison=pd.DataFrame({'features':['bias']+list(x_train.columns),
                                  'simple_model':w_sk,'ridge_wts':[ridge_final.intercept_]+list(ridge_final.coef_)})

In [ ]:
ridge_wt_comparison['suppression_ratio_l2']=ridge_wt_comparison['simple_model']/ridge_wt_comparison['ridge_wts']

In [ ]:
ridge_wt_comparison

you can make prediction with the fitted model in a similar manner 

In [ ]:
ridge_final.predict(x_train)

if you want to make prediction on test, data, transform it first with the datapipe we had fitted earlier for the same data

now lets see how $l_1$ penalty affects things

In [ ]:
from sklearn.linear_model import Lasso


In [ ]:
alphas_lasso=np.linspace(1,100,100)
params_lasso={'alpha':alphas_lasso}

In [ ]:
lr_lasso=Lasso()
gs_lasso=GridSearchCV(lr_lasso,
               param_grid=params_lasso,
               cv=10,
               scoring='neg_mean_absolute_error',
               verbose=20,
               n_jobs=-1)

In [ ]:
gs_lasso.fit(x_train,y_train)

In [ ]:
mt.report(gs_lasso.cv_results_,5)

you can see here that the best value comes at the left edge, we can probably improve our results by expanding our experiment values on that side

In [ ]:
lr_lasso=Lasso()
alphas_lasso=np.linspace(0,2,100)
params_lasso={'alpha':alphas_lasso}
gs_lasso=GridSearchCV(lr_lasso,
               param_grid=params_lasso,
               cv=10,
               scoring='neg_mean_absolute_error',
               verbose=20,
               n_jobs=-1)
gs_lasso.fit(x_train,y_train)

In [ ]:
mt.report(gs_lasso.cv_results_,5)

now we got a value inbetween the range, we will consider this as our final model for lasso

In [ ]:
lasso_final=Lasso(**{'alpha': 0.020202020202020204})

In [ ]:
lasso_final.fit(x_train,y_train)

In [ ]:
ridge_wt_comparison['lasso_wts']=[lasso_final.intercept_]+list(lasso_final.coef_)

In [ ]:
ridge_wt_comparison

you can see that $l_1$ penalty has made many weights exactly zero, to count how many weights have been made exactly zero

In [ ]:
(lasso_final.coef_==0).sum()

we can actually remove those features and built the model without them 

# Data Prep for Logistic Regression

In [ ]:
bd_train=pd.read_csv(r'./bd_train.csv')

In [ ]:
def children_to_num(col):
    
    num_col=col.str.replace('Zero','0')
    num_col=num_col.str.replace('4+','4',regex=False)
    num_col=pd.to_numeric(num_col,errors='coerce')
    
    return num_col

def ab_to_num(col):
    
    col=col.str.replace('71+','71-71',regex=False)
    k=col.str.split('-',expand=True)
    
    for i in [0,1]:
        k[i]=pd.to_numeric(k[i],errors='coerce')
        
    num_col=0.5*(k[0]+k[1])
    
    return num_col

def fi_to_num(col):
    
    col=col.replace({'<10,000, >= 8,000':9000, '>=35,000':35000, '<25,000, >=22,500':23750,
       '<20,000, >=17,500':18750, '<12,500, >=10,000':11250, '<30,000, >=27,500':28750,
       '<27,500, >=25,000':26250, '<17,500, >=15,000':16250, '<15,000, >=12,500':13750,
       '<22,500, >=20,000':21250,'< 4,000': 4000, '< 8,000, >= 4,000':6000})
    num_col=pd.to_numeric(col,errors='coerce')
    
    return num_col

simple_numeric_cols=['year_last_moved','Average.Credit.Card.Transaction', 'Balance.Transfer',
      'Term.Deposit', 'Life.Insurance', 'Medical.Insurance',
      'Average.A.C.Balance', 'Personal.Loan', 'Investment.in.Mutual.Fund',
      'Investment.Tax.Saving.Bond', 'Home.Loan', 'Online.Purchase.Amount','Investment.in.Commudity',
      'Investment.in.Equity', 'Investment.in.Derivative',
      'Portfolio.Balance']

cat_to_dummies_cols=['status' , 'occupation' , 'occupation_partner' , 'home_status', 'self_employed',
'self_employed_partner','TVarea','gender','region']

custom_function_cols={'children':children_to_num,'age_band':ab_to_num,'family_income':fi_to_num}

dp=mt.DataPipe(simple_num=simple_numeric_cols,
                     cat_to_dummies=cat_to_dummies_cols,
                     custom_func_dict=custom_function_cols)

dp.fit(bd_train)

x_train=dp.transform(bd_train)

y_train=(bd_train['Revenue.Grid']==1).astype(int)

## Logistic regression with gradient descent

you can use following functions to implement gradient descent version of parameter estimation for logistic regression, few things to keep in mind 

* standardize your data before using gradient descent with any optimizer you have in mind 
* you can `de-standardize` your estimates of weights thus obtained using the function `convert_to_non_standardized_weights` that we wrote earlier

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def mypred(x, w):
    """
    Logistic regression prediction (sigmoid function).
    """
    z = x @ w
    y_hat = sigmoid(z)
    return y_hat

def myerror(y, x, w):
    """
    Computes the prediction errors for logistic regression.
    """
    y_hat = mypred(x, w)
    errors = y - y_hat
    return errors

def mycost(y, x, w, alpha=0.1, penalty='none'):
    """
    Computes the cost function for logistic regression with either no penalty, L1 or L2 regularization.

    Parameters:
    - y: actual values
    - x: input data
    - w: weights
    - alpha: regularization strength
    - penalty: 'none' for no regularization, 'l1' for Lasso, or 'l2' for Ridge

    Returns:
    - cost: the regularized cost function value
    """
    m = x.shape[0]
    y_hat = mypred(x, w)

    # Logistic loss (cross-entropy loss)
    log_loss = -np.mean(y * np.log(y_hat + 1e-15) + (1 - y) * np.log(1 - y_hat + 1e-15))

    # Apply either L1 or L2 penalty, or no penalty
    if penalty == 'none':
        total_cost = log_loss
    elif penalty == 'l1':
        l1_penalty = np.sum(np.abs(w))
        total_cost = log_loss + alpha * l1_penalty / m
    elif penalty == 'l2':
        l2_penalty = np.sum(w ** 2)
        total_cost = log_loss + alpha * l2_penalty / (2 * m)
    else:
        raise ValueError("Invalid penalty type. Use 'none', 'l1', or 'l2'.")

    return total_cost

def gradient(y, x, w, alpha=0.1, penalty='none'):
    """
    Computes the gradient for logistic regression with either no penalty, L1 or L2 regularization.

    Parameters:
    - y: actual values
    - x: input data
    - w: weights
    - alpha: regularization strength
    - penalty: 'none' for no regularization, 'l1' for Lasso, or 'l2' for Ridge

    Returns:
    - grad: the gradient of the cost function with no regularization, L1, or L2 regularization
    """
    m = x.shape[0]
    y_hat = mypred(x, w)

    # Gradient of the logistic loss
    grad = -x.T @ (y - y_hat) / m

    # Apply regularization if specified
    if penalty == 'none':
        total_grad = grad
    elif penalty == 'l1':
        l1_grad = np.sign(w)  # Subgradient for L1
        total_grad = grad + alpha * l1_grad / m
    elif penalty == 'l2':
        l2_grad = 2 * w
        total_grad = grad + alpha * l2_grad / m
    else:
        raise ValueError("Invalid penalty type. Use 'none', 'l1', or 'l2'.")

    return total_grad


## Logistic Regression with sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
params_logr={'class_weight':['balanced',None],
       'penalty':['l1','l2'] ,# this is not 11 eleven, its L1 [el-one] in lower case 
        'C':[.0001,.0005,.001,.005,.01,.05,0.1,2,5,10]}

In [ ]:
logr=LogisticRegression(solver='liblinear')

In [ ]:
gs_logr=GridSearchCV(logr,
               param_grid=params_logr,
               scoring='roc_auc', # scoring here is roc_auc for its a binary classification problem
               cv=10,
               n_jobs=-1,
               verbose=20)

In [ ]:
gs_logr.fit(x_train,y_train)

In [ ]:
mt.report(gs_logr.cv_results_,5)

In [ ]:
logr_final=LogisticRegression(solver='liblinear',**{'C': 0.05, 'class_weight': 'balanced', 'penalty': 'l1'})

In [ ]:
logr_final.fit(x_train,y_train)

## predict probabilities

In [ ]:
# by default fitted model predicts probabilities for all the classes, and use the function predict_proba

In [ ]:
logr_final.predict_proba(x_train)

In [ ]:
# in order to understand which set of probabilities belong to what class look at this attribute
logr_final.classes_

In [ ]:
# first probability belongs to class 0 and second to class 1 for each obs

In [ ]:
class_1_probs=logr_final.predict_proba(x_train)[:,1]

## Predicting Hard Classes

In [ ]:
# to go to hard classes from these probabilities we need to find a proper threshold on the probabilities 
# lets find cutoff on the basis of KS , you can replicate the same with F_1 score also

In [ ]:
real=y_train
score=logr_final.predict_proba(x_train)[:,1]

In [ ]:
cutoffs=np.linspace(0.001,0.999,999)

In [ ]:
# we will calculate TP,TN,FP,FN for each cutoff and find corresponding KS value
# we select the ideal cutoff for which ks is maximum
# if there are multiple winners , we will simply go with the first one
all_ks=[]

for cutoff in cutoffs:
    
    # note that for each cutoff hard class predictions can be different
    
    predicted=(score>cutoff).astype(int) # this converts the True/False to 1/0
    
    TP=((real==1)&(predicted==1)).sum()
    FP=((real==0)&(predicted==1)).sum()
    TN=((real==0)&(predicted==0)).sum()
    FN=((real==1)&(predicted==0)).sum()
    
    P=TP+FN
    N=TN+FP
    
    ks=(TP/P)-(FP/N)
    
    all_ks.append(ks)

In [ ]:
max(all_ks)

In [ ]:
selected_cutoff=cutoffs[all_ks==max(all_ks)][0]

In [ ]:
hard_class_preds=(logr_final.predict_proba(x_train)[:,1]>selected_cutoff).astype(int)

In [ ]:
hard_class_preds